In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import learning.min_regret_strategy_pb2
import experimental.pokerbots.bin_centers_pb2
import numpy as np
import experimental.pokerbots.pokerbot as pokerbot
import experimental.pokerbots.pokerbots_scrape as pbs
import experimental.pokerbots.hand_evaluator_python as hep
import pandas as pd
import os

import importlib
importlib.reload(pokerbot)
importlib.reload(pbs)

In [ ]:
CLUSTER_PATH = '/home/erick/code/robot/experimental/pokerbots/cluster_hands'
# flop_data = pd.read_pickle(os.path.join(CLUSTER_PATH, 'flop_data.p'))
turn_data = pd.read_pickle(os.path.join(CLUSTER_PATH, 'turn_data_1000.p'))
river_data = pd.read_pickle(os.path.join(CLUSTER_PATH, 'river_data_1000.p'))

In [ ]:
strategies = {}
training_path = '/tmp/pokerbots_1000_bins_discount/'
for file in os.listdir(training_path):
    idx = int(file[-12:-3])
    if idx % 1000000 != 0:
        continue
    with open(os.path.join(training_path, file), 'rb') as file_in:
        s = learning.min_regret_strategy_pb2.MinRegretStrategy()
        s.ParseFromString(file_in.read())
    
    strategies[idx] = s

In [ ]:
strategy_file = '/tmp/pokerbots_1000_bins_discount/pokerbot_checkpoint_100000000.pb'
with open(strategy_file, 'rb') as file_in:
    strategy = learning.min_regret_strategy_pb2.MinRegretStrategy()
    strategy.ParseFromString(file_in.read())
bin_centers_file = '/home/erick/code/robot/experimental/pokerbots/bin_centers_1000.pb'
with open(bin_centers_file, 'rb') as file_in:
    per_turn_bin_centers = experimental.pokerbots.bin_centers_pb2.PerTurnBinCenters()
    per_turn_bin_centers.ParseFromString(file_in.read())

In [ ]:
PAGE_LIMIT = 5
session_value = pbs.load_session_value()
rows = pbs.pull_rows(PAGE_LIMIT, session_value)

In [ ]:
latest_games = rows[rows['your_bot'] == rows.loc[0, 'your_bot']]

In [ ]:
latest_games

In [ ]:
games = []
for i, row in latest_games.iterrows():
    games.append(pbs.get_game_log(row, session_value))

In [ ]:
len(games)

In [ ]:
def filter_games(predicate, games):
    filtered_games = []
    for game in games:
        filtered_games.append(game._replace(rounds = list(filter(predicate, game.rounds))))
        
    return filtered_games

In [ ]:
large_hands = filter_games(lambda game : abs(game.A_delta) > 300, games)

In [ ]:
def print_game(game_round):
    print(f'Delta A({game_round.A_delta}) B({game_round.B_delta})')
    print(f'A {game_round.A_hand} B {game_round.B_hand}')
    for round_num, (board, actions) in enumerate(zip(game_round.board_states, game_round.round_actions)):
        if board:
            print('Board:', board)
        for i, a in enumerate(actions):
            player = a[0]
            hand = game_round.A_hand if player == 'A' else game_round.B_hand
            action_queue = actions[:i]
            if round_num == 0:
                action_queue = action_queue[2:]
            
            infoset_actions = []
            for action in action_queue:
                if action[1] == 'calls':
                    infoset_actions.append('Call')
                if action[1] == 'checks':
                    infoset_actions.append('Check')
                if action[1] in ['raises', 'bets']:
                    infoset_actions.append('RaisePot' if action[2] < 100 else 'AllIn')                        
                
            betting_round = round_num
            if round_num > 2 and round_num < len(game_round.board_states) - 1:
                betting_round = 2
            elif round_num > 2 and round_num == len(game_round.board_states) -1:
                betting_round = 3
            
            infoset_id = pokerbot.compute_infoset_id(infoset_actions, betting_round, hand, board, per_turn_bin_centers)
            print(a, infoset_actions, infoset_id[0], hex(infoset_id[0]))
    

In [ ]:
def find_infoset(infoset_id, strategy=strategy):
    return list(filter(lambda infoset: infoset.id_num == infoset_id, strategy.infoset_counts))[0]

In [ ]:
print(large_hands[6].a_bot)

In [ ]:
print(large_hands[6].a_bot)
print_game(large_hands[6].rounds[0])

In [ ]:
infoset_id = 0x200000096
infoset = find_infoset(infoset_id)
print(infoset)
norm_strategy = np.array(infoset.strategy_sum) / np.sum(infoset.strategy_sum)
print(norm_strategy)

In [ ]:
bucket_hands = turn_data.loc[turn_data['cluster_idx'] == (infoset_id & 0xFFFF)]

In [ ]:
bucket_hands.mean(numeric_only=True)

In [ ]:
result = hep.evaluate_strength_potential(''.join(['Kh', '7h']), ''.join(['Kd', '6h', 'Td', '5c', '4h']), 2, timeout_s=1.0)

In [ ]:
result

In [ ]:
print(result.strength, result.negative_potential, result.positive_potential)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(131)
_ = plt.hist(bucket_hands['strength'],bins=40)
plt.subplot(132)
_ = plt.hist(bucket_hands['neg_pot'],bins=40)
plt.subplot(133)
_ = plt.hist(bucket_hands['pos_pot'],bins=40)
plt.tight_layout()

In [ ]:
def plot_infoset_history(infoset_id, strategies):
    num_strategies = len(strategies)
    data = np.zeros((num_strategies, 5))
    idxs = []
    for i, key in enumerate(sorted(strategies.keys())):
        all_strategies = strategies[key]
        infoset_counts = list(filter(lambda x : x.id_num == infoset_id, all_strategies.infoset_counts))[0]
        idxs.append(key)
        strategy = np.array(infoset_counts.strategy_sum)
        strategy = strategy / np.sum(strategy)
        data[i, :] = strategy
    plt.plot(idxs, data, label=['Fold', 'Check', 'Call', 'RaisePot', 'AllIn'])
    plt.legend()
    plt.title(f'Infoset Id: {hex(infoset_id)}')
    plt.xlabel('Strategy')
    plt.ylabel('Training Iteration')

In [ ]:
plt.figure()
plot_infoset_history(infoset_id, strategies)

In [ ]:
counts = [x.iter_count for x in strategy.infoset_counts]